In [ ]:
!pip install eventregistry
!pip install spacy
!pip install fsspec
!pip install gcsfs 
!pip install text2emotion
!pip install newsapi-python
!pip install cloudstorage

     |████████████████████████████████| 51 kB 445 kB/s 
  Created wheel for eventregistry: filename=eventregistry-8.10-py3-none-any.whl size=63178 sha256=9982e912f72bc75cc20ce9353d27319d53abaa3665be283a237b8969c23bfad0
  Stored in directory: /root/.cache/pip/wheels/85/87/51/89006a9990978feb1f136874d2ce40df94b38a7f1c7ecfb271
Successfully built eventregistry
     |████████████████████████████████| 118 kB 15.0 MB/s 
     |████████████████████████████████| 1.3 MB 16.0 MB/s 
     |████████████████████████████████| 294 kB 41.9 MB/s 
     |████████████████████████████████| 142 kB 44.1 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 184 kB 25.8 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=7fa214f72a9d350bfb2412f8630d50ee5adf51bcfd2e29e3308532b3d04f8160
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji
     |████████

In [ ]:
from eventregistry import *
import json, os, sys
import cloudstorage as gcs
import pandas as pd
from io import StringIO
from pandas.io.json import json_normalize
from google.colab import auth
from google.cloud import bigquery
from google.cloud import storage
auth.authenticate_user()
import statistics
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from newsapi import NewsApiClient
import datetime
import text2emotion as te
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import imaplib
import base64
import os
import email
import requests
stop_words = set(stopwords.words("english"))
tokenizer = RegexpTokenizer(r'\w+')
nltk.download('vader_lexicon')
sns.set(style='darkgrid', context='talk', palette='Dark2')
print('Authenticated')

KeyboardInterrupt: ignored

In [ ]:
projectId = "BQ_Project"
bqclient = bigquery.Client(project=projectId)
storage_client = storage.Client(project=projectId)
bucket = storage_client.get_bucket("GCS_BUCKET_NAME")
forecast_URL = "WEATHER_API"
current_weather_url = "WEATHER_API"

In [ ]:
dataLocation = 'GCS_BUCKET_LOCATION'
current = date.today()
days = datetime.timedelta(90)
new_date = current - days
new_date = new_date.strftime("%Y-%m-%d")
current = current.strftime("%Y-%m-%d")
def createSentimentDataset(dataLocation, apiCall):
  #Declarations
  urls = []
  new_list = []

  df = pd.read_csv(dataLocation)
  df = df.drop([3,4,7])
  for x in df['URL'].iloc[0:]:
    urls.append(x)
    
  for x in urls:
    q = QueryArticlesIter(
        keywords = QueryItems.OR(["Covid-19", "Business","Politics","Travel","Lifestyle",'Weather',"Sustainable","New Zealand"]),
        sourceUri= apiCall.getSourceUri(x),
        sourceLocationUri = apiCall.getLocationUri("New Zealand"),
        dateStart = new_date,
        dateEnd = current)

    print("Number of results: %d" % q.count(er))

    for art in q.execQuery(apiCall, sortBy = "rel", maxItems = 2000, returnInfo = ReturnInfo( 
        storyInfo=StoryInfoFlags(title=True), 
        articleInfo=ArticleInfoFlags(concepts=True, categories=True, duplicateList=True,socialScore =True,bodyLen=30000),
        conceptInfo=ConceptInfoFlags(trendingScore=True),
        sourceInfo = SourceInfoFlags(socialMedia=True,ranking=True))):
        new_list.append(art)
  print("Dataset Completed!")

  return new_list

In [ ]:
def normalizeDataset(df):

  new_df = pd.json_normalize(df)
  new_df = pd.DataFrame(data=df)
  new_df = new_df.rename(columns={'source.uri':'source_uri','source.dataType':'source_dataType','source.title':'source_title','shares.facebook':'shares_facebook','shares.pinterest':'shares_pinterest',
                          'source.ranking.importanceRank':'source_ranking_importanceRank','source.ranking.alexaGlobalRank':'source_ranking_alexaGlobalRank','source.ranking.alexaCountryRank':'source_ranking_alexaCountryRank',
                          'source.socialMedia.twitter':'source_socialMedia_twitter','source.socialMedia.facebook':'source_socialMedia_facebook'})
  new_df.sort_values(by=['relevance'],ascending=False)

  #Drop duplicate urls
  new_df = new_df.drop_duplicates(subset="url")
  #Store normalized dataset in GCS

  new_df['Count'] = 1
  new_df['body'].to_csv('GCS_BUCKET_LOCATION')
  new_df['url'].to_csv('GCS_BUCKET_LOCATION')


  #Store raw data in BigQuery
  new_df.to_gbq('baconator.raw_data', 'omega-cider-297120', if_exists='replace'
                 )
  print("Data Pushed!")
  return new_df

In [ ]:
def createPolarityAnalysis(normalizeDf):
  def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

  normalizeDf['sentiment'] = normalizeDf['body'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
  normalizeDf['sentiment'] = normalizeDf['sentiment'].apply(getAnalysis)

  normalizeDf = pd.DataFrame({'Date':normalizeDf['date'],'Sentiments':normalizeDf['sentiment'],'Body':normalizeDf['body'],'URL':normalizeDf['url'],"Relevance":normalizeDf['relevance'],'Count':normalizeDf['Count']})

  #Store Analysis in BigQuery
  normalizeDf.to_gbq('baconator.textblob', projectId, if_exists='replace'
                 )
  print("Segment Pushed!")

  return normalizeDf

In [ ]:
def aggregateData(datorama_df,social_studio):

  datorama_df = datorama_df[['Date','Sentiment','Count']]
  social_studio= social_studio[['Date','Sentiment','Count']]

  frames = [datorama_df,social_studio]
  final_df = pd.concat(frames)

  final_df = final_df.groupby([final_df['Date'],final_df['Sentiment']]).sum().reset_index()

  final_df.to_gbq('baconator.aggregated_data',projectId,if_exists='replace')

  return "Data Pushed"

In [1]:
datorama_datalocation = "GCS_BUCKET_LOCATION"
social_studio_location ="GCS_BUCKET_LOCATION"
def datoramaDataPull(dataLocation):
  def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

  datorama_df = pd.read_csv(dataLocation)
  datorama_df = datorama_df.dropna()
  datorama_df = datorama_df.rename(columns={'Competitive Page Content Text':'Body','Competitive Page Content Date Formatted':'Date','Competitive Page Content Url':'URL','Page Content Count':"Count"})
  datorama_df['Polarity'] = datorama_df['Body'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
  datorama_df['Sentiment'] = datorama_df['Polarity'].apply(getAnalysis)
  datorama_df['Date'] = pd.to_datetime(datorama_df['Date'])
  datorama_df = datorama_df[(datorama_df['Date'] > '2021-05-01') & (datorama_df['Date'] <= current)]
  datorama_df['Date'] = datorama_df['Date'].dt.strftime("%Y-%m-%d")
  
  datorama_df.to_gbq('baconator.datorama_textblob',projectId,if_exists='replace')
  print("Data Pushed")

  return datorama_df

In [2]:
def socialStudioPull(dataLocation):
  def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

  datorama_df = pd.read_csv(dataLocation)
  datorama_df = datorama_df.dropna()
  datorama_df = datorama_df.rename(columns={'Social Mention Content':'Body','Social Mention Publish Date':'Date','Social Mention Url':'URL','Social Mention Count':"Count"})
  datorama_df['Polarity'] = datorama_df['Body'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
  datorama_df['Sentiment'] = datorama_df['Polarity'].apply(getAnalysis)
  datorama_df['Date'] = pd.to_datetime(datorama_df['Date'])
  datorama_df = datorama_df[(datorama_df['Date'] > '2021-05-01') & (datorama_df['Date'] <= current)]
  datorama_df['Date'] = datorama_df['Date'].dt.strftime("%Y-%m-%d")

  return datorama_df

In [3]:
def weatherForecast(url):
  forecase_response = requests.get(url)
  forecast_dict = forecase_response.json()
  forecast_df = pd.json_normalize(forecast_dict)

  #Unnest data
  forecast_df = forecast_df.explode('forecast.forecastday')
  forecast_df = pd.json_normalize(forecast_df['forecast.forecastday'])
  forecast_df = forecast_df[['date','day.avgtemp_c','day.condition.text']]
  forecast_df = forecast_df.rename(columns={'date':"Date"})
  return forecast_df

In [4]:

def getCurrentWeather(url):
  
  current_response = requests.get(current_weather_url)
  current_dict = forecase_response.json()
  current_df = pd.json_normalize(forecast_dict)

  # #Unnest data
  current_df = current_df[['current.last_updated','current.temp_c','current.condition.text']]
  current_df['current.last_updated'] = pd.to_datetime(current_df['current.last_updated'])
  current_df['current.last_updated'] = current_df['current.last_updated'].dt.strftime("%Y-%m-%d")
  current_df = current_df.rename(columns={'current.last_updated':"Date","day.avgtemp_c":'Temperature','day.condition.text':'Weather'})
  return current_df

In [ ]:
def combineDataForecast(current,forecast):

  mergedf = pd.merge(current,forecast, on="Date")
  mergedf = mergedf.rename(columns={'current.temp_c':"current_temp_c","current.condition.text":'current_condition','day.avgtemp_c':'forecast_temp_c',"day.condition.text":'forecast_condition'})
  mergedf.to_gbq('BQ_DATASET','BQ_PROJECT',if_exists='replace')

  return mergedf

In [ ]:
combineDataForecast(current_weather,forecast_df)

1it [00:05,  5.39s/it]


,Date,current_temp_c,current_condition,forecast_temp_c,forecast_condition
0,2021-08-11,15.0,Partly cloudy,11.6,Patchy rain possible


# **Call Functions** 

**Important Note: Only run the *createSentimentDataset* once as we only have 2000 tokens request available on NewsAPI**